In [0]:
%run "./15 Intro to Delta Tables"

Crear VIEWS persistentes

In [0]:
CREATE OR REPLACE VIEW people_view AS
SELECT * FROM people_delta
WHERE age >= 30;


In [0]:

SELECT * FROM people_view;

Crear VIEW TEMPORAL

In [0]:
CREATE OR REPLACE TEMP VIEW people_temp_view AS
SELECT name, age, TRUE AS emitio_voto FROM people_delta
WHERE SUBSTRING(name,1,1) = 'M' ;  -- Ejemplo de filtro adicional



In [0]:
SELECT * FROM people_temp_view;

View Global `Temporal` - NO SOPORTADO EN Serverless

In [0]:
CREATE OR REPLACE GLOBAL TEMP VIEW people_oro_global_view AS
SELECT id, name, age FROM people_delta
WHERE age >= 65;

CREATE TABLE AS (CTAS)


In [0]:
CREATE TABLE IF NOT EXISTS ciudadano_oro
USING DELTA
AS
SELECT *, TRUE AS ciudadano_oro
FROM people_delta
WHERE age >= 65;

In [0]:

SELECT * FROM ciudadano_oro;

CHECKS y CONSTRAINTS

In [0]:
ALTER TABLE people_delta
ADD CONSTRAINT chk_age_nonneg CHECK (age >= 0);


In [0]:
ALTER TABLE people_delta
ALTER COLUMN name SET NOT NULL;

In [0]:
INSERT INTO people_delta
VALUES
(200,"Juan Perez", -1)

CTAS con particiones

In [0]:
CREATE TABLE IF NOT EXISTS people_by_age_group
USING DELTA
COMMENT "Tabla particionada por grupos de edad"
PARTITIONED BY (age_group)
AS
SELECT
  name,
  age,
  CASE
    WHEN age < 18 THEN 'Adolescente'
    WHEN age < 35 THEN 'Adulto Joven'
    WHEN age < 65 THEN 'Adulto'
    ELSE 'Adulto Mayor'
  END AS age_group
FROM people_delta;

In [0]:
DESCRIBE EXTENDED people_by_age_group

In [0]:
DESCRIBE DETAIL people_by_age_group

In [0]:
SHOW PARTITIONS people_by_age_group;


In [0]:
EXPLAIN EXTENDED
SELECT COUNT(*) 
FROM people_by_age_group 
WHERE age_group = 'Adulto Mayor';

In [0]:
DROP TABLE IF EXISTS people_nopart_age_group;
CREATE TABLE IF NOT EXISTS people_nopart_age_group
USING DELTA
COMMENT "Tabla NO particionada por grupos de edad"
AS
SELECT
  name,
  age,
  CASE
    WHEN age < 18 THEN 'Adolescente'
    WHEN age < 35 THEN 'Adulto Joven'
    WHEN age < 65 THEN 'Adulto'
    ELSE 'Adulto Mayor'
  END AS age_group
FROM people_delta;

In [0]:
SHOW PARTITIONS people_nopart_age_group;


In [0]:
EXPLAIN EXTENDED
SELECT COUNT(*) 
FROM people_nopart_age_group
WHERE age_group = 'Adulto Mayor';

COPIAR TABLAS
DEEP CLONE

Deep clone: copia datos, esquema y metadatos (historia)


In [0]:
DROP TABLE IF EXISTS people_delta_clone_deep;
CREATE TABLE people_delta_clone_deep
DEEP CLONE people_delta;

In [0]:
DESCRIBE HISTORY people_delta_clone_deep

In [0]:
DESCRIBE HISTORY people_delta

SHALLOW CLONE

In [0]:
DROP TABLE IF EXISTS people_delta_clone_shallow;
CREATE TABLE people_delta_clone_shallow
SHALLOW CLONE people_delta;

In [0]:
DESCRIBE HISTORY people_delta_clone_shallow

In [0]:
DESCRIBE HISTORY people_delta

Ahora insertamos un registro en la tabla original

In [0]:
INSERT INTO people_delta
SELECT max(id)+1, 'Juan DeepClone', 15
FROM people_delta;

Y revisamos de nuevo los conteos:

In [0]:
SELECT 'original'      AS source, COUNT(*) AS cnt FROM people_delta
UNION ALL
SELECT 'deep_clone', COUNT(*) FROM people_delta_clone_deep
UNION ALL
SELECT 'shallow_clone', COUNT(*) FROM people_delta_clone_shallow;

In [0]:
DESCRIBE DETAIL people_delta;


In [0]:
DESCRIBE DETAIL people_delta_clone_deep;


In [0]:
DESCRIBE DETAIL people_delta_clone_shallow;
